# Serve multiple LoRA adapters efficiently on SageMaker - vLLM

In this tutorial, we will learn how to serve many Low-Rank Adapters (LoRA) on top of the same base model efficiently on the same GPU. In order to do this, we'll deploy a [vLLM serving-based container](https://docs.vllm.ai/en/latest/serving/deploying_with_docker.html) to SageMaker Hosting. 

These are the steps we will take:

1. [Setup our environment](#setup)
2. [Build a new vLLM container image compatible with SageMaker, push it to Amazon ECR](#container)
3. [Download adapters from the HuggingFace Hub and upload them to S3](#download_adapter)
4. [Build LoRA modules manifest file](#manifest)
5. [Deploy the extended vLLM container to SageMaker](#deploy)
6. [Compare outputs of the base model and the adapter model](#compare)
7. [Benchmark our deployed endpoint under different traffic patterns - same adapter, and random access to many adapters](#benchmark)


## What is vLLM? 

vLLM is a fast and easy-to-use library for LLM inference and serving. It supports most state-of-the art LLM serving optimizations, such as PagedAttention, FlashAttention, continuous batching and more. 

Recently vLLM added support for efficient multi-LoRA serving, with one of the key features being support for different LoRA ranks in the same batch. This is important for users that tune each adapter's rank to its specific task and dataset to get the best overall performance (although the need for this is not definitive, see [here](https://arxiv.org/abs/2402.09353)).

You can read more about vLLM and its multi-LoRA serving feature [here](https://docs.vllm.ai/en/latest/models/lora.html).

<a id="setup"></a>
## Setup our environment 

In [ ]:
!pip install -U boto3 sagemaker huggingface_hub --quiet

In [ ]:
import sagemaker
import boto3
sess = sagemaker.Session()

# sagemaker session bucket -> used for uploading data, models and logs
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region = sess.boto_region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {region}")

<a id="container"></a>
## Build a new vLLM container image compatible with SageMaker, push it to Amazon ECR

This example includes a `Dockerfile` and `sagemaker_entrypoint.sh` in the `sagemaker_vllm` directory. Building this new container image makes vLLM compatible with SageMaker Hosting, namely launching the server on port 8080 via the container's `ENTRYPOINT` instruction, and changing the relevant server routes from the original `/ping` and `/v1/completions` to `/health` and `/invocations` . [Here](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html#your-algorithms-inference-code-run-image) you can find the basic interfaces required to adapt any container for deployment on Sagemaker Hosting.

We also make sure all relevant server configuration arguments (such as enabling the LoRA serving feature, LoRA module paths, etc.) are configurable via environment variables, so that our container entrypoint can be parametrized at runtime by SageMaker. vLLM does not support env var-based server config by default. You can find all possible configuration parameters within vllm [Server args](https://github.com/vllm-project/vllm/blob/865732342b4e3b8a4ef38f28a2a5bdb87cf3f970/vllm/entrypoints/openai/cli_args.py#L25) and [AsyncEngine args](https://github.com/vllm-project/vllm/blob/865732342b4e3b8a4ef38f28a2a5bdb87cf3f970/vllm/engine/arg_utils.py#L12); if you want to expose other parameters, add them to the Dockerfile, and make sure to pass them when launching the SageMaker Endpoint, as we will see in the next sections. A relevant one to expose for larger models would be [--tensor-parallel-size](https://github.com/vllm-project/vllm/blob/865732342b4e3b8a4ef38f28a2a5bdb87cf3f970/vllm/engine/arg_utils.py#L164C30-L164C52).

Let's analyze the Dockerfile and entrypoint script to understand how easy it is to adapt any serving framework (and vLLM in particula) to run on SageMaker Real-Time Hosting.


In [ ]:
!pygmentize sagemaker_vllm/sagemaker_entrypoint.sh
!printf "\n\n\nEnd of entrypoint script ----------------"

Here are the relevant things to note from the previous cell output:
1. we force the server to run on port 8080, as required by SageMaker
2. selected base model is parameterized with `HF_MODEL_ID` env var
3. maximum allowed sequence lenght (input+output) is parameterized with `MAX_MODEL_LEN` env var; this is an important parameter, as many models' max len is larger than what a single A10G can hold, which would cause the server to error and exit at startup
4. maximum number of LoRA adapters that can run within the same batch on the GPU are parameterized with `MAX_GPU_LORAS` env var; the default in vLLM is 1, which would provide poor performance. To define an appropriate value for this parameter, you should take into consideration the total memory of the GPU you will be deploying on, memory required for each adapter, and the expected input/output lengths of the incoming payloads
5. maximum number of LoRA adapters that can be offloaded to CPU memory (RAM) for quick hotswapping is parameterized with `MAX_CPU_LORAS`. To define an appropriate value for this parameter, you should take into consideration the total RAM available in the instance type you will deploy on, and memory required for each adapter
6. maximum number of sequences that can be processed per iteration is parameterized with `MAX_NUM_SEQS`; it's important to tailor this to the GPU being used, as some GPU memory is pre-allocated based on its value
7. whether to enforce eager or not is parameterizes with `ENFORCE_EAGER`; by default vLLM captures the model for CUDA graphs which reduces its latency, but it also consumes an extra 1-3GB of memory, so it can be turned off by enforcing eager mode
8. the names (invocation target ids) and local paths for all LoRA adapters and their artifacts are listed within a manifest file (we will construct it later according to vLLM's `--lora-modules` [arg specification](https://docs.vllm.ai/en/latest/models/lora.html#serving-lora-adapters)), the name and directory of which we pass via the `LORA_MODULES_MANIFEST_FILE` and `MODEL_DIR` env vars

**Why do we have to pass all local directories for adapter artifacts in 8.?** --> At the time of writing, vLLM's LoRA serving feature does not allow for dynamic downloads of LoRA adapters from S3 or HF Hub as they are invoked. All adapters must be present locally on the underlying instance that the server runs on. That does not mean you have to include all the adapter artifacts in your container image, as this would be very rigid and unfriendly for image reusability. We will show you how downloading adapters from S3 can be done dynamically before the server starts up with the help of Sagemaker in the next sections.

**Why a manifest file instead of just another environment variable?** --> SageMaker enforces the length of the json encoded env vars dictionary that is passed to be under 1024 characters. This might not be enough to build the `--lora-modules` argument, especially as the number of adapters (i.e. modules) grows. With this in mind, we will build a manifest file that is downloaded and read into a variable before the vLLM server is started, sidestepping this limitation.


<div class="alert alert-block alert-info">
⚠️ The above approach is specific to the latest version of the vLLM container at the time of writing, and will likely change with updates to vLLM.
</div>

In [ ]:
!pygmentize sagemaker_vllm/Dockerfile
!printf "\n\n\nEnd of Dockerfile ----------------"

In the output of the above cell, you can see that we:
* fix to a specific vLLM container version, which we pass when we build the image
* replace the `/health` and `/v1/completions` server routes by `/ping` and `/invocations` in the main server launch script, as required by SageMaker
* copy our entrypoint script to the container, and set it as the ENTRYPOINT command, as required by SageMaker

! NOTE !: if you change the vLLM base container version, check to make sure the string replacements above still work as intended, and the path to the main server launch script still holds

We are good to go! We build the new container image and push it to a new ECR repository. Note SageMaker [supports private Docker registries](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-containers-inference-private.html) as well.

In [ ]:
%%bash -s {region}
algorithm_name="sagemaker-vllm"  # name of your algorithm
tag="v0.3.3"
region=$1

account=$(aws sts get-caller-identity --query Account --output text)

image_uri="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:${tag}"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" --region $region > /dev/null
fi

cd sagemaker_vllm/ && docker build --build-arg VERSION=$tag -t ${algorithm_name}:${tag} .

# Authenticate Docker to an Amazon ECR registry
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com

# Tag the image
docker tag ${algorithm_name}:${tag} ${image_uri}

# Push the image to the repository
docker push ${image_uri}

# Save image name to tmp file to use when deploying endpoint
echo $image_uri > /tmp/image_uri

<a id="download_adapter"></a>
## Download adapter from HuggingFace Hub and push it to S3

We are going to simulate storing our adapter weights on S3, and having SageMaker download them upfront when we provision the endpoint. This enables most scenarios, including deployment after you’ve finetuned your own adapters and pushed them to S3, as well as securing deployments with no internet access inside your VPC, as detailed in this [blog post](https://www.philschmid.de/sagemaker-llm-vpc#2-upload-the-model-to-amazon-s3).

We first download an adapter trained with Mistral Instruct v0.1 as the base model to a local directory. This particular adapter was trained on GSM8K, a grade school math dataset.

In [ ]:
from pathlib import Path
from huggingface_hub import snapshot_download

HF_MODEL_ID = "vineetsharma/qlora-adapter-Mistral-7B-Instruct-v0.1-gsm8k"
# create model dir
model_dir = Path('mistral-adapter')
model_dir.mkdir(exist_ok=True)

# Download model from Hugging Face into model_dir
snapshot_download(
    HF_MODEL_ID,
    local_dir=str(model_dir), # download to model dir
    local_dir_use_symlinks=False, # use no symlinks to save disk space
    revision="main", # use a specific revision, e.g. refs/pr/21
    cache_dir='/home/ec2-user/SageMaker/.cache/'
)

We copy this same adapter `n_adapters` times to different S3 prefixes in our SageMaker session bucket, simulating a large number of adapters we want to serve on the same endpoint and underlying GPU. We name the last prefix directory (leaf directory) as integer indexes, but you can change this to reflect the task or name of each adapter. 

In [ ]:
import os

s3 = boto3.client('s3')

def upload_folder_to_s3(local_path, s3_bucket, s3_prefix):
    for root, dirs, files in os.walk(local_path):
        for file in files:
            local_file_path = os.path.join(root, file)
            s3_object_key = os.path.join(s3_prefix, os.path.relpath(local_file_path, local_path))
            s3.upload_file(local_file_path, s3_bucket, s3_object_key)

# Upload the folder n_adapters times under different prefixes
n_adapters=50
base_prefix = 'vllm/mistral-adapters'
for i in range(1, n_adapters+1):
    prefix = f'{base_prefix}/{i}'
    upload_folder_to_s3(model_dir, sagemaker_session_bucket, prefix)
    print(f'Uploaded folder to S3 with prefix: {prefix}')

<a id="manifest"></a>
## Build LoRA modules manifest file

Now, we build the manifest file. The `--lora-modules` arg syntax to follow is `MODULE_NAME_1=LOCAL_DIRECTORY_1 MODULE_NAME_2=LOCAL_DIRECTORY_2 ...`. 

To maintain consistency and ease of management, we want our module names (which we will invoke later) to match the names of the leaf directories on S3 that hold each adapter artifact. In order to do this, we list the contents of our `base_prefix` directory on S3, parse the leaf directory names, and use them to build the `--lora-modules` argument. In this case, the leaf names will be integers in the range of 1 to `n_adapters`; however, the following cells can be reused for any S3 prefix that holds directories with LoRA artifacts.  

In [ ]:
def list_s3_leaf_directories(bucket_name, prefix):
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter='/')

    leaf_directories = []
    for prefix in response.get('CommonPrefixes', []):
        directory_path = prefix['Prefix']
        directory_name = directory_path.rstrip('/').rsplit('/', 1)[-1]  # Extract the last part of the path, the leaf directory
        leaf_directories.append(directory_name)

    return leaf_directories


bucket_name = sagemaker_session_bucket
prefix = base_prefix+'/'
leaf_directories = list_s3_directories(bucket_name, prefix)
print(leaf_directories)

Now, we build the manifest file and upload it to S3

In [ ]:
lora_modules_manifest_file = "lora_modules.txt"

with open(lora_modules_manifest_file, "w") as file:
    vllm_arg_pattern = " ".join([f"{leaf_dir}=/opt/ml/model/{leaf_dir}/" for leaf_dir in leaf_directories])
    file.write(vllm_arg_pattern)

# Upload to S3
s3.upload_file(lora_modules_file, sagemaker_session_bucket, f'{base_prefix}/{lora_modules_manifest_file}')

We can verify the file was uploaded to the correct directory (our base_prefix) on the following cell. If there is no output, the file is not where it should be, and you should verify the previous steps.

In [ ]:
! aws s3 ls "s3://$sagemaker_session_bucket/$base_prefix/" | grep "$lora_modules_manifest_file"

<a id="deploy"></a>
## Deploy SageMaker endpoint


Now we deploy a SageMaker endpoint, pointing to our `base_prefix` as the `model_data` parameter.

Let's dissect what is happening here:
* as explained in the SageMaker [docs](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html#your-algorithms-inference-code-load-artifacts), SageMaker downloads model artifacts under the provided `S3URI` to the `/opt/ml/model` directory; your container has read-only access to this directory
* by specifying that our data is in an `S3Prefix` and `CompressionType` is `None`, you do not need to tar.gz the `base_prefix` directory; SageMaker will download all the files and directories in our `base_prefix` in uncompressed format, replicating the S3 directory structure (i.e. the `base_prefix` dir structure will match the `/opt/ml/model` dir structure). This is why we pass `/opt/ml/model` as the `MODEL_DIR` in the next cell, and why we placed the lora manifest file in the root of the `base_prefix` 


In [ ]:
import json
import datetime

from sagemaker import Model
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# Retrieve image_uri from tmp file
image_uri = !cat /tmp/image_uri
# Increased health check timeout to give time for model download
health_check_timeout = 800
# Endpoint configs
number_of_gpu = 1
instance_type = "ml.g5.xlarge"
endpoint_name = sagemaker.utils.name_from_base("sm-vllm")

# Env vars required for server launch
config = {
  'MODEL_DIR': '/opt/ml/model', # root dir for adatper dirs and manifest file
  'LORA_MODULES_MANIFEST_FILE': lora_modules_manifest_file, # manifest file name
  'HF_MODEL_ID': "mistralai/Mistral-7B-Instruct-v0.1", # model_id from hf.co/models
  'MAX_MODEL_LEN': json.dumps(4096),  # max length of input text
  'MAX_GPU_LORAS': json.dumps(20), # max number of adapters usable in single batch
  'MAX_CPU_LORAS': json.dumps(50), # max number of adapters that can be held in CPU mem
  'MAX_NUM_SEQS': json.dumps(100), # max number of sequences per iteration
  'ENFORCE_EAGER': json.dumps(False), # whether to turn off CUDA graphs and enforce eager mode (saves GPU mem) 
}

# Create SM Model, pass in model data as a whole prefix of uncompressed model artifacts
vllm_model = Model(
    image_uri=image_uri[0],
    model_data={
        'S3DataSource':{
            'S3Uri': f's3://{sagemaker_session_bucket}/{base_prefix}/',
            'S3DataType': 'S3Prefix',
            'CompressionType': 'None'}},
    env=config,
    role=role,
)

vllm_predictor = vllm_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

In [ ]:
# You can reinstantiate the Predictor object if you restart the notebook or Predictor is None
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
endpoint_name = endpoint_name

vllm_predictor = Predictor(
    endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

<a id="compare"></a>
## Invoke base model and adapter, compare outputs

We can invoke the base Mistral model, as well as any of the adapters downloaded to our endpoint! vLLM will take care of downloading them, continuously batch requests for different adapters, and manage DRAM and RAM by loading/offloading adapters.

Let’s inspect the difference between the base model’s response and the adapter’s response:

In [ ]:
prompt = '[INST] Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? [/INST]'

payload_base = {
    "model": "mistralai/Mistral-7B-Instruct-v0.1",
    "prompt": prompt,
    "max_tokens": 64,
    "temperature":0
}


payload_adapter = {
    "model": "1",
    "prompt": prompt,
    "max_tokens": 64,
    "temperature":0
}

response_base = vllm_predictor.predict(payload_base)
response_adapter = vllm_predictor.predict(payload_adapter)

print(f'Base model output:\n-------------\n {response_base["choices"][0]["text"]}')
print(f'\nAdapter output:\n-------------\n {response_adapter["choices"][0]["text"]}')

You can also check out the full details of the response

In [ ]:
response_adapter

<a id="benchmark"></a>
## Benchmark single adapter vs. random access to adapters



First, we individually call each of the adapters in sequence, to make sure they are previously loaded to either GPU or CPU memory. We want to exclude disk read latency from the benchmark metrics.

In [ ]:
from tqdm import tqdm

for i in tqdm(range(1,n_adapters+1)):
    payload_adapter = {
        "model": str(i),
        "prompt": prompt,
        "max_tokens": 64,
        "temperature":0
    }
    vllm_predictor.predict(payload_adapter)

Now we are ready to benchmark. For the single adapter case, we invoke the adapter `total_requests` times from `num_threads` concurrent clients.

For the multi-adapter case, we invoke a random adapter from any of the clients, until all adapters have been invoked `total_requests//num_adapters` times.

In [ ]:
# Adjust if you run into connection pool errors
# import botocore

# Configure botocore to use a larger connection pool
# config = botocore.config.Config(max_pool_connections=100)

In [ ]:
import threading
import time
import random


# Configuration
total_requests = 300
num_adapters = 50
num_threads = 20  # Adjust based on your system capabilities


# Shared lock and counters for # invocations of each adapter 
adapter_counters = [total_requests // num_adapters] * num_adapters
counters_lock = threading.Lock()

def invoke_adapter(aggregate_latency, single_adapter=False):
    global total_requests
    latencies = []
    while True:
        with counters_lock:
            if single_adapter:
                adapter_id = 1
                if total_requests > 0:
                    total_requests -= 1
                else:
                    break
            else:
                # Find an adapter that still needs to be called
                remaining_adapters = [i for i, count in enumerate(adapter_counters) if count > 0]
                if not remaining_adapters:
                    break
                adapter_id = random.choice(remaining_adapters) + 1
                adapter_counters[adapter_id - 1] -= 1

        prompt = '[INST] Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May? [/INST]'

        payload_adapter = {
            "model": str(adapter_id),
            "prompt": prompt,
            "max_tokens": 64,
            "temperature":0
        }
        start_time = time.time()
        response_adapter = vllm_predictor.predict(payload_adapter)
        latency = time.time() - start_time
        latencies.append(latency)

    aggregate_latency.extend(latencies)

def benchmark_scenario(single_adapter=False):
    threads = []
    all_latencies = []
    start_time = time.time()

    for _ in range(num_threads):
        thread_latencies = []
        all_latencies.append(thread_latencies)
        thread = threading.Thread(target=invoke_adapter, args=(thread_latencies, single_adapter))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    total_latency = sum([sum(latencies) for latencies in all_latencies])
    total_requests_made = sum([len(latencies) for latencies in all_latencies])
    average_latency = total_latency / total_requests_made
    throughput = total_requests_made / (time.time() - start_time)

    print(f"Total Time: {time.time() - start_time}s")
    print(f"Average Latency: {average_latency}s")
    print(f"Throughput: {throughput} requests/s")

# Run benchmarks
print("Benchmarking: Single Adapter Multiple Times")
benchmark_scenario(single_adapter=True)

print("\nBenchmarking: Multiple Adapters with Random Access")
benchmark_scenario()


<a id="cleanup"></a>
## Cleanup endpoint resources

In [ ]:
vllm_predictor.delete_endpoint()